## Modules

In [ ]:
#General packages
from collections import defaultdict
import matplotlib.pyplot as plt #only for visualisation
import networkx as nx #only for visualisation
import pyomo.environ as pe
import pyomo.opt as po
import math
from functools import reduce
import operator as op
import logging #only for managing logging
import itertools

#Packages specifically for positioning of nodes for visualisation
import grandalf as grand
from grandalf.layouts import SugiyamaLayout
from grandalf.graphs import Graph, Vertex, Edge

## System settings

In [ ]:
logging.getLogger('pyomo.core').setLevel(logging.ERROR) #Don't show warnings

## Functions

In [ ]:
#Create product function (for Python versions < 3.8)
def prod(iterable):
    return reduce(op.mul, iterable, 1)

## IT-Network Graph

In [ ]:
#Define graph nodes
nodes_network = set(range(1,27))
s_network = 1 #artificial start node
t_network = 26 #artificial end node
targets_network = {8,9,10,11,12,16,17,18,19,20,21,22,23,24,25}#set of goal nodes

#Define graph edges
edges_network = {(2,6),(2,7),(3,15),(4,8),(5,9),(5,10),(6,13),(6,14),(6,15),(7,13),(7,14),(7,15),(13,16),(14,16),(15,17),
    (16,9),(16,19),(16,20),(16,21),(17,18),(18,21),(21,22),(21,23),(1,2),(1,3),(1,4),(1,5),(8,26),(9,26),(10,26),(11,26),
    (12,26),(16,26),(17,26),(18,26),(19,26),(20,26),(21,26),(22,26),(23,26),(24,26),(25,26)} #network edges
art_edges_s_network = {(1,2),(1,3),(1,4),(1,5)}#set of edges from artificial start node
art_edges_t_network = {(8,26),(9,26),(10,26),(11,26),(12,26),(16,26),(17,26),(18,26),(19,26),(20,26),(21,26),(22,26),
    (23,26),(24,26),(25,26)} #set of edges from artificial end node

#Define graph attributes
att_cost = {(2,6): 2,(2,7): 2,(3,15): 3,(4,8): 4,(5,9): 5,(5,10): 5,(6,13): 1,(6,14): 1,(6,15): 1,(7,13): 1,(7,14): 1,
    (7,15): 2,(13,16): 3,(14,16): 3,(15,17): 1,(16,9): 1,(16,19): 1,(16,20): 1,(16,21): 1,(17,18): 3,(18,21): 1,(21,22): 3,
    (21,23): 3,(1,2): 0,(1,3): 0,(1,4): 0,(1,5): 0,(8,26): 0,(9,26): 0,(10,26): 0,(11,26): 0,(12,26): 0,(16,26): 0,
    (17,26): 0,(18,26): 0,(19,26): 0,(20,26): 0,(21,26): 0,(22,26): 0,(23,26): 0,(24,26): 0,(25,26): 0} #attack cost from
    #each edge
def_cost = {(2,6): 1,(2,7): 1,(3,15): 1,(4,8): 1,(5,9): 1,(5,10): 1,(6,13): 1,(6,14): 1,(6,15): 1,(7,13): 1,(7,14): 1,
    (7,15): 1,(13,16): 1,(14,16): 1,(15,17): 1,(16,9): 1,(16,19): 1,(16,20): 1,(16,21): 1,(17,18): 1,(18,21): 1,(21,22): 1,
    (21,23): 1,(1,2): 100,(1,3): 100,(1,4): 100,(1,5): 100,(8,26): 100,(9,26): 100,(10,26): 100,(11,26): 100,(12,26): 100,
    (16,26): 100,(17,26): 100,(18,26): 100,(19,26): 100,(20,26): 100,(21,26): 100,(22,26): 100,(23,26): 100,(24,26): 100,
    (25,26): 100} #defence cost for each edge

## System Process Graph

In [ ]:
#Define graph nodes
nodes_sysprocess = set(range(1,49))
s_sysprocess = 1 #artificial start node
t_sysprocess = 36 #artificial end node
sys_input = {2, 3, 4, 8, 9, 10, 11, 16, 17, 18, 19, 20, 21, 37, 41} #set of subsystem input nodes 
sys_output = {5, 6, 7, 12, 13, 14, 15, 22, 23, 24, 25, 26, 27, 38, 42} #set of subsystem output nodes
capability_input = {28, 29, 30, 31, 39} #set of capability input nodes
capability_output = {32, 33, 34, 35, 40} #set of capability output nodes
process = {43, 44, 45, 46, 47, 48} #set of process nodes

#Define graph edges
edges_sysprocess={(2,5),(3,6),(4,7),(8,12),(9,13),(10,14),(11,15),(16,22),(17,23),(18,24),(19,25),(20,26),(21,27),
    (37,38),(41,42),(28,32),(29,33),(30,34),(31,35),(39,40),(43,46),(44,47),(45,48),(5,8),(5,9),(5,10),(5,11),(5,17),(5,18),
    (5,19),(5,20),(5,21),(6,8),(6,9),(6,10),(6,11),(6,16),(6,17),(6,18),(6,19),(7,31),(12,16),(12,17),(12,20),(13,18),
    (13,19),(13,21),(14,19),(15,16),(15,17),(15,18),(15,19),(15,21),(15,37),(22,28),(23,28),(23,29),(24,29),(25,28),(25,29),
    (25,30),(26,30),(27,30),(38,39),(42,31),(32,43),(33,44),(34,44),(35,43),(35,44),(35,45),(40,45),(1,2),(1,3),(1,4),
    (1,41),(46,36),(47,36),(48,36)} #system and process edges
targets_sysprocess = {(2,5),(3,6),(4,7),(8,12),(9,13),(10,14),(11,15),(16,22),(17,23),(18,24),(19,25),(20,26),(21,27),
    (37,38),(41,42)} #set of goal edges
art_edges_s_sysprocess = {(1,2),(1,3),(1,4),(1,41)} #set of edges from artificial start node
art_edges_t_sysprocess = {(46,36),(47,36),(48,36)} #set of edges from artificial end node

#Define graph attributes
capacity_sysprocess ={(2,5): 1,(3,6): 1,(4,7): 1,(8,12): 1,(9,13): 1,(10,14): 1,(11,15): 1,(16,22): 1,(17,23): 1,(18,24): 1,
    (19,25): 1,(20,26): 1,(21,27): 1,(37,38): 1,(41,42): 1,(28,32): 1,(29,33): 1,(30,34): 1,
    (31,35): 1,(39,40): 1,(43,46): math.inf,(44,47): math.inf,(45,48): math.inf,(5,8): 1,(5,9): 1,(5,10): 1,
    (5,11): 1,(5,17): 1,(5,18): 1,(5,19): 1,(5,20): 1,(5,21): 1,(6,8): 1,(6,9): 1,(6,10): 1,(6,11): 1,(6,16): 1,(6,17): 1,
    (6,18): 1,(6,19): 1,(7,31): 1,(12,16): 1,(12,17): 1,(12,20): 1,(13,18): 1,(13,19): 1,(13,21): 1,(14,19): 1,(15,16): 1,
    (15,17): 1,(15,18): 1,(15,19): 1,(15,21): 1,(15,37): 1,(22,28): 1,(23,28): 1,(23,29): 1,(24,29): 1,(25,28): 1,
    (25,29): 1,(25,30): 1,(26,30): 1,(27,30): 1,(38,39): 1,(42,31): 1,(32,43): 1,(33,44): 1,(34,44): 1,
    (35,43): 1,(35,44): 1,(35,45): 1,(40,45): 1,(1,2): 1,(1,3): 1,(1,4): 1,(1,41): 1,
    (46,36): math.inf,(47,36): math.inf,(48,36): math.inf} #Upper bound on flow/capacity of each edge

## Model parameters

In [ ]:
att_budget = 9 #total attack budget
def_budget = 3 #total defence budget

In [ ]:
#Network - System Process relation
graph_relation = {(2,5): 11,(3,6): 10,(4,7): 12,(8,12): 16,(9,13): 21,(10,14): 18,(11,15): 8,(16,22): 19,(17,23): 9,
                  (18,24): 22,(19,25): 17,(20,26): 20,(21,27): 23,(37,38): 25,(41,42): 24}#links target nodes in the
    #IT-network graph to target edges in the system process graph

## Graphs Visualisation

In [ ]:
#Define IT-network graph
network = nx.DiGraph()
network.add_nodes_from(list(nodes_network))
network.add_edges_from(list(edges_network))
pos_network = nx.spring_layout(network)#Messy positioning just for verification...

In [ ]:
#Create a cost tuple for visualisation
merged_cost = att_cost.copy()
for key in att_cost:
    merged_cost[key] = (att_cost[key],def_cost[key])

In [ ]:
#Plot the graph
fig, ax = plt.subplots(figsize=(12, 12))
nx.draw_networkx_nodes(network, pos=pos_network, ax=ax, node_color='lightgray',
                       edgecolors='black', node_size=2000)
nx.draw_networkx_labels(network, pos=pos_network, ax=ax, labels=dict(zip(nodes_network, nodes_network)),
                        font_size=20)
nx.draw_networkx_edges(network, pos=pos_network, ax=ax, node_size=2000, arrowsize=25)
nx.draw_networkx_edge_labels(network, pos=pos_network, ax=ax, edge_labels=merged_cost,
                             font_size=16, rotate=False)
plt.axis('off')
plt.show()

In [ ]:
#Determine initial positions for graph visualisation
def layered(G):
    V = []
    data_to_V = {}
    for x in G.nodes():
        vertex = Vertex(x)
        V.append(vertex)
        data_to_V[x] = vertex
    E = [Edge(data_to_V[xy[0]], data_to_V[xy[1]], data=xy) for xy in G.edges()]
    g = Graph(V, E)

    class defaultview(object):
        w, h = 10, 10
    for v in V: v.view = defaultview()

    sug = SugiyamaLayout(g.C[0])
    sug.init_all()#roots=[V[0]])
    sug.draw()#calculate positions

    pos = {v.data: (v.view.xy[0], v.view.xy[1]) for v in g.C[0].sV}#extracts the positions
    
    return pos

#Define system process graph and fix position of specific nodes
sysprocess = nx.DiGraph()
sysprocess.add_nodes_from(list(nodes_sysprocess))
sysprocess.add_edges_from(list(edges_sysprocess))
pos_sysprocess = layered(sysprocess)
pos_sysprocess[11]=(105,95)
pos_sysprocess[15]=(105,125)
pos_sysprocess[16]=(60,155)
pos_sysprocess[22]=(60,185)
pos_sysprocess[31]=(195,215)
pos_sysprocess[35]=(195,245)
pos_sysprocess[37]=(105,155)
pos_sysprocess[38]=(105,185)
pos_sysprocess[39]=(105,215)
pos_sysprocess[40]=(105,245)
pos_sysprocess[41]=(240,35)
pos_sysprocess[42]=(240,65)
pos_sysprocess[43]=(105,275)
pos_sysprocess[44]=(-45,275)
pos_sysprocess[45]=(30,275)
pos_sysprocess[46]=(105,305)
pos_sysprocess[47]=(-45,305)
pos_sysprocess[48]=(30,305)
pos_sysprocess[10]=(-15,95)
pos_sysprocess[14]=(-15,125)
pos_sysprocess[19]=(-45,155)
pos_sysprocess[25]=(-45,185)
pos_sysprocess[8]=(45,95)
pos_sysprocess[12]=(45,125)
pos_sysprocess[17]=(5,155)
pos_sysprocess[23]=(5,185)
pos_sysprocess[4]=(150,35)
pos_sysprocess[7]=(150,65)

In [ ]:
#Plot the graph
fig, ax = plt.subplots(figsize=(12, 24))
nx.draw_networkx_nodes(sysprocess, pos=pos_sysprocess, ax=ax, node_color='lightgray',
                       edgecolors='black', node_size=2000)
nx.draw_networkx_labels(sysprocess, pos=pos_sysprocess, ax=ax, labels=dict(zip(nodes_sysprocess, nodes_sysprocess)),
                        font_size=20)
nx.draw_networkx_edges(sysprocess, pos=pos_sysprocess, ax=ax, node_size=2000, arrowsize=25)
plt.axis('off')
plt.show()

## Attacker sub-problem model

In [ ]:
history_attack_plans = {}

In [ ]:
def attacker(nodes_network,edges_network,s_network,t_network,targets_network,att_budget,att_cost,nodes_sysprocess,
             edges_sysprocess,s_sysprocess,t_sysprocess,sys_input,sys_output,cap_input,cap_output,process,
             art_edges_t_sysprocess,capacity,targets_sysprocess,graph_relation,history_attack_plans,patched_edges = {}):  
    
    #Model object
    model = pe.ConcreteModel() #define name and type of model
    
    #Define model sets from earlier defined graphs
    model.nodes_network = pe.Set(initialize=nodes_network)
    model.edges_network = pe.Set(within=model.nodes_network*model.nodes_network, initialize=edges_network)
    model.targets_network = pe.Set(initialize=targets_network)
    model.nodes_sysprocess = pe.Set(initialize=nodes_sysprocess)
    model.edges_sysprocess = pe.Set(within=model.nodes_sysprocess*model.nodes_sysprocess, initialize=edges_sysprocess)
    model.sys_input = pe.Set(initialize=sys_input)
    model.sys_output = pe.Set(initialize=sys_output)
    model.cap_input = pe.Set(initialize=capability_input)
    model.cap_output = pe.Set(initialize=capability_output)
    model.process = pe.Set(initialize=process)
    model.art_t_sysprocess = pe.Set(initialize=art_edges_t_sysprocess)
    model.targets_sysprocess = pe.Set(initialize=targets_sysprocess)
    model.patched_edges = pe.Set(initialize=patched_edges)
    
    #Pre-calculated sets of incoming and outgoing network edges to speed up processing
    Vm_network = defaultdict(set)
    Vp_network = defaultdict(set)
    for (i, j) in edges_network:
        Vm_network[i].add(j)
        Vp_network[j].add(i)
        
    #Pre-calculated sets of incoming and outgoing sysprocess edges to speed up processing
    Vm_sysprocess = defaultdict(set)
    Vp_sysprocess = defaultdict(set)
    for (i, j) in edges_sysprocess:
        Vm_sysprocess[i].add(j)
        Vp_sysprocess[j].add(i)
    
    #Define model parameters from earlier defined graphs and pre-calculated sets
    model.Vm_network = pe.Param(model.nodes_network, initialize=Vm_network, default=set(), within=pe.Any)
    model.Vp_network = pe.Param(model.nodes_network, initialize=Vp_network, default=set(), within=pe.Any)
    model.Vm_sysprocess = pe.Param(model.nodes_sysprocess, initialize=Vm_sysprocess, default=set(), within=pe.Any)
    model.Vp_sysprocess = pe.Param(model.nodes_sysprocess, initialize=Vp_sysprocess, default=set(), within=pe.Any)
    model.capacity = pe.Param(model.edges_sysprocess, initialize=capacity_sysprocess)
    model.att_cost = pe.Param(model.edges_network, initialize=att_cost)
    model.graph_relation = pe.Param(model.targets_sysprocess, initialize=graph_relation)
    model.s_network = s_network
    model.t_network = t_network
    model.s_sysprocess = s_sysprocess
    model.t_sysprocess = t_sysprocess
    model.att_budget = att_budget
    model.bigm2 = len(model.targets_network)
    model.history_attack_plans = history_attack_plans
    
    #Define model variables
    model.x_network = pe.Var(model.edges_network, within=pe.NonNegativeIntegers) #attack indicator on each network edge
    model.y_network = pe.Var(model.edges_network, initialize=0, within=pe.Binary) #cumulative attack indicator on each
        #edge (sum of x over entire attack plan)
    model.x_sysprocess = pe.Var(model.edges_sysprocess, initialize=0, within=pe.Binary) #availability indicator on each
        #system process edge (linked to x_network and y_network)
    model.y_sysprocess = pe.Var(model.edges_sysprocess, within=pe.NonNegativeIntegers) #directional flow of system
        #performance on each system process edge
    model.b = pe.Var(model.targets_network, within=pe.Binary) #breach indicator on each network edge
    
    #Objective function z_AD (eq. 5.17)
    def best_attack(model):
        return sum(model.y_sysprocess[i, j] for (i, j) in model.art_t_sysprocess) - sum(2 * model.y_sysprocess[i, j] * \
            model.x_sysprocess[i, j] for i, j in model.edges_sysprocess)

    model.obj_best_attack = pe.Objective(sense=pe.minimize, rule=best_attack)#add objective to model
    
    #Constraints
    ##Flow balance network (eq. 5.9)
    def flow_balance_network(model, n):
        flow_in = sum(model.x_network[i, n] for i in model.Vp_network[n])
        flow_out = sum(model.x_network[n, j] for j in model.Vm_network[n])
        if n == model.s_network:
            constraint = (flow_out - flow_in == flow_out)
        elif n == model.t_network:
            constraint = (flow_out - flow_in == -flow_in)
        else:
            constraint = (flow_out - flow_in == 0)
        return constraint

    model.con_flow_balance_network = pe.Constraint(model.nodes_network, rule=flow_balance_network)#add constraint to model
    
    ##Breach target (eq. 5.10)
    def breach(model, g):
        return model.b[g] - sum(model.x_network[i, j] for (i, j) in model.edges_network if j in model.targets_network \
            and j == g) == 0

    model.con_breach = pe.Constraint(model.targets_network, rule=breach)#add constraint to model
    
    ##Attacker resources (eq. 5.11)
    def attack_resources(model):
        spending = sum(model.att_cost[i, j] * model.y_network[i, j] for (i, j) in model.edges_network)
        return spending <= model.att_budget

    model.con_attack_resources = pe.Constraint(rule=attack_resources)#Add constraint to model
    
    ##Attack indicator (eq. 5.12)
    def attack_indicator(model, i, j):
        return model.x_network[i, j] <= model.bigm2*model.y_network[i, j]

    model.con_attack_indicator = pe.Constraint(model.edges_network, rule=attack_indicator)#add constraint to model
    
    ##Conversion from network to sysprocess (eq. 5.14)
    def conversion(model, i, j):
        if (i, j) in model.targets_sysprocess:
            constraint = (model.x_sysprocess[i, j] == model.b[model.graph_relation[i, j]])
        else:
            constraint = (model.x_sysprocess[i, j] == 0)
        return constraint    
            
    model.con_conversion = pe.Constraint(model.edges_sysprocess,rule=conversion)#Add constraint to model

    ##Correction to suppress tendency to switch on unconstrained y's on the artificial edges. Cosmetic solver issue,
    #not part of the actual model.
    def flow_balance_network_cor(model, i, j):
        return model.y_network[i, j] <= model.x_network[i, j]

    model.con_flow_balance_network_cor = pe.Constraint(model.edges_network, rule=flow_balance_network_cor)#add constraint 
    #to model
    
    ##Flow balance system/processes (eq. 5.15)
    def flow_balance_sysprocess(model, n):
        flow_in = sum(model.y_sysprocess[i, n] for i in model.Vp_sysprocess[n])#Sum of all incoming arcs
        flow_out = sum(model.y_sysprocess[n, j] for j in model.Vm_sysprocess[n])#Sum of all outgoing arcs
        prod_in = prod(model.y_sysprocess[i, n] for i in model.Vp_sysprocess[n])#Product of all incoming arcs
        att_cor = 1- sum(model.x_sysprocess[n, j] for j in model.Vm_sysprocess[n])#Correction factor for attacked arcs
        if n == model.s_sysprocess:
            constraint = (flow_out - flow_in == len(model.Vm_sysprocess[n]))
        elif n in model.sys_input:
            constraint = (flow_out - flow_in == -(prod_in * att_cor * (flow_in-1)) - ((1- prod_in * att_cor) * flow_in))
        elif n in model.sys_output:
            constraint = (flow_out - flow_in == flow_in * (len(model.Vm_sysprocess[n])-1))
        elif n in model.process:
            constraint = (flow_out - flow_in == prod_in - flow_in)
        elif n in model.cap_input:
            constraint = (flow_out - flow_in == -(prod_in * (flow_in-1)) - ((1- prod_in) * flow_in))
        elif n in model.cap_output:
            constraint = (flow_out - flow_in == flow_in * (len(model.Vm_sysprocess[n])-1))
        elif n == model.t_sysprocess:
            constraint = (flow_out - flow_in == -flow_in)
        return constraint

    model.con_flow_balance_sysprocess = pe.Constraint(model.nodes_sysprocess, rule=flow_balance_sysprocess)#Add constraint to model

    ##Arc capacity (eq. 5.7)
    def arc_capacity(model, i, j):
        return (model.y_sysprocess[i, j] <= model.capacity[i, j])
    
    model.con_arc_capacity = pe.Constraint(model.edges_sysprocess, rule=arc_capacity)#Add constraint to model
    
    ##Patched (eq. 5.19)
    def patched(model, i, j):
        if (i, j) in model.patched_edges:
            constraint = (model.y_network[i, j] == 0)         
        else:
            constraint = pe.Constraint.Skip
        return constraint
    
    model.con_patched = pe.Constraint(model.edges_network, rule=patched)#Add constraint to model
    
    ##Avoid duplicate attacks (eq. 5.23)
    def duplicates(model):
        if model.history_attack_plans:
            for k in model.history_attack_plans:
                constraint = (sum((((1 - model.history_attack_plans[k][i, j]) * model.y_network[i, j]) for (i, j) in \
                model.edges_network)) + sum((model.history_attack_plans[k][i, j] * (1 -model.y_network[i, j])) for (i, j) \
                in model.edges_network) >= 1)    
        else:
            constraint = pe.Constraint.Skip
        return constraint
    
    model.con_duplicates = pe.Constraint(rule=duplicates)#Add constraint to model
         
    #Solver
    solver = po.SolverFactory('scip', executable="[DRIVE AND PATH TO EXECUTABLE]/scipampl.exe")
    result = solver.solve(model, tee=False)
    
    return model.obj_best_attack(),model.x_sysprocess,model.y_sysprocess,model.x_network,model.y_network,model.b

In [ ]:
result,x_sysprocess,y_sysprocess,x_network,y_network,b = attacker(nodes_network,edges_network,s_network,t_network,
                        targets_network,att_budget,att_cost,nodes_sysprocess,edges_sysprocess,s_sysprocess,t_sysprocess,
                        sys_input,sys_output,capability_input,capability_output,process,art_edges_t_sysprocess,
                        capacity_sysprocess,targets_sysprocess,graph_relation,history_attack_plans)

In [ ]:
print('z_ad value')
print('Performance: ',result)
print('x_sysprocess')
for i in x_sysprocess:
    print('Edge: ',i,' Operabel: ',x_sysprocess[i].value)
print('y_sysprocess')
for i in y_sysprocess:
    print('Edge: ',i,' Operabel: ',y_sysprocess[i].value)
print('x_network values')
for i in x_network:
    print('Edge: ',i,' Times attacked: ',x_network[i].value)
print('y_network values')
for i in y_network:
    print('Edge: ',i,' Attacked: ',y_network[i].value)
print('b values')
for i in b:
    print('Target node: ',i,' Breached: ',b[i].value)

## Visualize Solution with NetworkX

In [ ]:
#Prepare network defence visualisation
patched_edges = {}
patched = list(patched_edges)

#Prepare network attack visualisation
y_network_edge_list_0 = [(i,j) for i,j in y_network if y_network[i,j].value == 0]
y_network_node_list_0 = list(set(itertools.chain.from_iterable(y_network_edge_list_0)))
y_network_edge_list_1 = [(i,j) for i,j in y_network if y_network[i,j].value == 1]
y_network_node_list_1 = list(set(itertools.chain.from_iterable(y_network_edge_list_1)))

#Prepare sysprocess visualisation
y_sysprocess_edge_list_0 = [(i,j) for i,j in y_sysprocess if y_sysprocess[i,j].value == 0]
y_sysprocess_node_list_0 = list(set(itertools.chain.from_iterable(y_sysprocess_edge_list_0)))
y_sysprocess_edge_list_1 = [(i,j) for i,j in y_sysprocess if y_sysprocess[i,j].value >= 1]
y_sysprocess_node_list_1 = list(set(itertools.chain.from_iterable(y_sysprocess_edge_list_1)))

In [ ]:
node_colors_network = ['lightblue' if i in y_network_node_list_1 
                       else 'lightgray' for i in network.nodes()]
edge_colors_network = ['blue' if (i, j) in y_network_edge_list_1
                       else 'green' if (i, j) in patched
                       else 'black' for (i, j) in network.edges()]
edge_widths_network = [2 if (i, j) in y_network_edge_list_1
                       else 2 if (i, j) in patched
                       else 1 for (i, j) in network.edges()]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
nx.draw_networkx_nodes(network, pos=pos_network, ax=ax, node_color=node_colors_network,
                       edgecolors='black', node_size=2000)
nx.draw_networkx_labels(network, pos=pos_network, ax=ax, labels=dict(zip(nodes_network, nodes_network)),
                        font_size=20)
nx.draw_networkx_edges(network, pos=pos_network, ax=ax, node_size=2000, arrowsize=25, edge_color=edge_colors_network, width=edge_widths_network)
plt.axis('off')
plt.show()

In [ ]:
node_colors = ['lightblue' if i in y_sysprocess_node_list_1 else 'lightgray'
               for i in sysprocess.nodes()]
edge_colors = ['blue' if (i, j) in y_sysprocess_edge_list_1 else 'black'
               for (i, j) in sysprocess.edges()]
edge_widths = [2 if (i, j) in y_sysprocess_edge_list_1 else 1
               for (i, j) in sysprocess.edges()]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 24))
nx.draw_networkx_nodes(sysprocess, pos=pos_sysprocess, ax=ax, node_color=node_colors,
                       edgecolors='black', node_size=2000)
nx.draw_networkx_labels(sysprocess, pos=pos_sysprocess, ax=ax, labels=dict(zip(nodes_sysprocess, nodes_sysprocess)),
                        font_size=20)
nx.draw_networkx_edges(sysprocess, pos=pos_sysprocess, ax=ax, node_size=2000, arrowsize=25, edge_color=edge_colors, width=edge_widths)
plt.axis('off')
plt.show()

## Master problem model

In [ ]:
history_attack_edges = {(16, 26),(6, 15),(14, 16),(5, 10),(17, 18),(1, 3),(15, 17),(6, 14),(4, 8),(3, 15),(1, 2),(1, 5),
                        (17, 26),(13, 16),(8, 26),(10, 26),(6, 13),(18, 21),(21, 23),(23, 26),(1, 4),(2, 6)}

In [ ]:
def defender(nodes_network,edges_network,s_network,t_network,targets_network,att_cost,nodes_sysprocess,
             edges_sysprocess,s_sysprocess,t_sysprocess,sys_input,sys_output,cap_input,cap_output,process,
             art_edges_t_sysprocess,art_edges_s_network,capacity,targets_sysprocess,graph_relation,
             history_attack_edges,def_cost,def_budget):  
    #Model object
    model = pe.ConcreteModel()
    
    #Data preparation
    history_attack_nodes = set(itertools.chain.from_iterable(history_attack_edges))
    
    #Sets
    model.nodes_network = pe.Set(initialize=nodes_network)
    model.edges_network = pe.Set(within=model.nodes_network*model.nodes_network, initialize=edges_network)
    model.targets_network = pe.Set(initialize=targets_network)
    model.nodes_sysprocess = pe.Set(initialize=nodes_sysprocess)
    model.edges_sysprocess = pe.Set(within=model.nodes_sysprocess*model.nodes_sysprocess, initialize=edges_sysprocess)
    model.sys_input = pe.Set(initialize=sys_input)
    model.sys_output = pe.Set(initialize=sys_output)
    model.cap_input = pe.Set(initialize=capability_input)
    model.cap_output = pe.Set(initialize=capability_output)
    model.process = pe.Set(initialize=process)
    model.art_s_network = pe.Set(initialize=art_edges_s_network)
    model.art_t_sysprocess = pe.Set(initialize=art_edges_t_sysprocess)
    model.targets_sysprocess = pe.Set(initialize=targets_sysprocess)
    model.history_attack_nodes = pe.Set(initialize=history_attack_nodes)
    model.history_attack_edges = pe.Set(within=model.history_attack_nodes*model.history_attack_nodes, \
                                        initialize=history_attack_edges)
    
    #Pre-calculated sets of incoming and outgoing network edges to speed up processing
    Vm_network = defaultdict(set)
    Vp_network = defaultdict(set)
    for (i, j) in edges_network:
        Vm_network[i].add(j)
        Vp_network[j].add(i)
        
    #Pre-calculated sets of incoming and outgoing sysprocess edges to speed up processing
    Vm_sysprocess = defaultdict(set)
    Vp_sysprocess = defaultdict(set)
    for (i, j) in edges_sysprocess:
        Vm_sysprocess[i].add(j)
        Vp_sysprocess[j].add(i)
        
    #Pre-calculated sets of incoming and outgoing attack selection edges to speed up processing
    Vm_cut = defaultdict(set)
    Vp_cut = defaultdict(set)
    for (i, j) in history_attack_edges:
        Vm_cut[i].add(j)
        Vp_cut[j].add(i)
    
    #Parameters
    model.Vm_network = pe.Param(model.nodes_network, initialize=Vm_network, default=set(), within=pe.Any)
    model.Vp_network = pe.Param(model.nodes_network, initialize=Vp_network, default=set(), within=pe.Any)
    model.Vm_sysprocess = pe.Param(model.nodes_sysprocess, initialize=Vm_sysprocess, default=set(), within=pe.Any)
    model.Vp_sysprocess = pe.Param(model.nodes_sysprocess, initialize=Vp_sysprocess, default=set(), within=pe.Any)
    model.Vm_cut = pe.Param(model.nodes_network, initialize=Vm_cut, default=set(), within=pe.Any)
    model.Vp_cut = pe.Param(model.nodes_network, initialize=Vp_cut, default=set(), within=pe.Any)
    model.capacity = pe.Param(model.edges_sysprocess, initialize=capacity_sysprocess)
    model.att_cost = pe.Param(model.edges_network, initialize=att_cost)
    model.def_cost = pe.Param(model.edges_network, initialize=def_cost)
    model.graph_relation = pe.Param(model.targets_sysprocess, initialize=graph_relation)
    model.s_network = s_network
    model.t_network = t_network
    model.s_sysprocess = s_sysprocess
    model.t_sysprocess = t_sysprocess
    model.def_budget = def_budget
    
    #Variables
    model.x_network = pe.Var(model.edges_network, within=pe.NonNegativeIntegers)
    model.x_sysprocess = pe.Var(model.edges_sysprocess, initialize=0, within=pe.Binary)
    model.y_sysprocess = pe.Var(model.edges_sysprocess, within=pe.NonNegativeIntegers)
    model.b = pe.Var(model.nodes_network, initialize=0, within=pe.Binary)
    model.w = pe.Var(model.edges_network, initialize=0, within=pe.Binary)
    
    #Objective function z_DAD (eq. 5.17)
    def best_defense(model):
        return sum(model.y_sysprocess[i, j] for (i, j) in model.art_t_sysprocess) - sum(2 * model.y_sysprocess[i, j] * \
                    model.x_sysprocess[i, j] for i, j in model.edges_sysprocess)

    model.obj_best_defense = pe.Objective(sense=pe.maximize, rule=best_defense)#Add objective to model
    
    #Constraints 
    ##Breach target (5.21, changed from 5.10)
    def breach(model, n):
        return ((1 - prod(1- model.x_network[i, n] for i in model.Vp_cut[n])) == model.b[n])

    model.con_breach = pe.Constraint(model.nodes_network, rule=breach)#Add constraint to model
    
    ##Attack indicator (5.22, changed from 5.12)
    def attack_indicator(model, n, j):
        prod_in = prod(model.w[i, n] for i in model.Vp_cut[n])
        if (n, j) in model.art_s_network:
            constraint = (model.x_network[n, j] == 1)
        else:
            constraint = (model.x_network[n, j] == model.b[n] * (1 - model.w[n, j]))
        return constraint
        
    model.con_attack_indicator = pe.Constraint(model.history_attack_edges, rule=attack_indicator)
    
    ##Conversion from network to sysprocess (eq. 5.14) 
    def conversion(model, i, j):
        if (i, j) in model.targets_sysprocess:
            constraint = (model.x_sysprocess[i, j] == model.b[model.graph_relation[i, j]])
        else:
            constraint = (model.x_sysprocess[i, j] == 0)
        return constraint    
            
    model.con_conversion = pe.Constraint(model.edges_sysprocess,rule=conversion)#Add constraint to model
       
    ##Flow balance  (eq. 5.15)
    def flow_balance_sysprocess(model, n):
        flow_in = sum(model.y_sysprocess[i, n] for i in model.Vp_sysprocess[n])#Sum of all incoming arcs
        flow_out = sum(model.y_sysprocess[n, j] for j in model.Vm_sysprocess[n])#Sum of all outgoing arcs
        prod_in = prod(model.y_sysprocess[i, n] for i in model.Vp_sysprocess[n])#Product of all incoming arcs
        att_cor = 1- sum(model.x_sysprocess[n, j] for j in model.Vm_sysprocess[n])#Correction factor for attacked arcs
        if n == model.s_sysprocess:
            constraint = (flow_out - flow_in == len(model.Vm_sysprocess[n]))
        elif n in model.sys_input:
            constraint = (flow_out - flow_in == -(prod_in * att_cor * (flow_in-1)) - ((1- prod_in * att_cor) * flow_in))
        elif n in model.sys_output:
            constraint = (flow_out - flow_in == flow_in * (len(model.Vm_sysprocess[n])-1))
        elif n in model.process:
            constraint = (flow_out - flow_in == prod_in - flow_in)
        elif n in model.cap_input:
            constraint = (flow_out - flow_in == -(prod_in * (flow_in-1)) - ((1- prod_in) * flow_in))
        elif n in model.cap_output:
            constraint = (flow_out - flow_in == flow_in * (len(model.Vm_sysprocess[n])-1))
        elif n == model.t_sysprocess:
            constraint = (flow_out - flow_in == -flow_in)
        return constraint

    model.con_flow_balance_sysprocess = pe.Constraint(model.nodes_sysprocess, rule=flow_balance_sysprocess)#Add constraint to model

    ##Arc capacity (eq. 5.7)
    def arc_capacity(model, i, j):
        return (model.y_sysprocess[i, j] <= model.capacity[i, j])
    
    model.con_arc_capacity = pe.Constraint(model.edges_sysprocess, rule=arc_capacity)#Add constraint to model
    
    ##Defence resources (eq. 5.18)
    def defence_resources(model):
        spending_def = sum(model.def_cost[i, j] * model.w[i, j] for (i, j) in model.edges_network)
        return (spending_def <= model.def_budget)
    
    model.con_defence_resources = pe.Constraint(rule=defence_resources)#Add constraint to model
    
    ##Subgraph selection (eq. 5.20)
    def selection(model, i, j):
        if (i, j) not in model.history_attack_edges:
            constraint = (model.x_network[i, j] + model.w[i, j] == 0)
        else:
            constraint = pe.Constraint.Skip
        return constraint
    
    model.con_selection = pe.Constraint(model.edges_network, rule=selection)
      
    #Solver
    solver = po.SolverFactory('scip', executable="[DRIVE AND PATH TO EXECUTABLE]/scipampl.exe")
    result = solver.solve(model, tee=False)
    
    return model.obj_best_defense(),model.x_sysprocess,model.y_sysprocess,model.x_network,model.b,model.w

In [ ]:
result,x_sysprocess,y_sysprocess,x_network,b,w = defender(nodes_network,edges_network,s_network,t_network,targets_network,att_cost,nodes_sysprocess,
                        edges_sysprocess,s_sysprocess,t_sysprocess,sys_input,sys_output,capability_input,capability_output,
                        process,art_edges_t_sysprocess,art_edges_s_network,capacity_sysprocess,targets_sysprocess,graph_relation,
                        history_attack_edges,def_cost,def_budget)

In [ ]:
print('z_dad value')
print('Performance: ',result)
print('x_network values')
for i in x_network:
    print('Edge: ',i,' Times attacked: ',x_network[i].value)
print('b values')
for i in b:
    print('Target node: ',i,' Breached: ',b[i].value)
print('w values')
for i in w:
    print('Node: ',i,' Patched: ',w[i].value)
print('x_sysprocess')
for i in x_sysprocess:
    print('Edge: ',i,' Operabel: ',x_sysprocess[i].value)
print('y_sysprocess')
for i in y_sysprocess:
    print('Edge: ',i,' Operabel: ',y_sysprocess[i].value)

## Visualize Solution with NetworkX

In [ ]:
#Prepare graph visualisation
greyed_out = [(i,j) for (i,j) in x_network if (i,j) not in history_attack_edges]

#Prepare network attack visualisation
x_network_edge_list_0 = [(i,j) for i,j in x_network if x_network[i,j].value == 0]
x_network_node_list_0 = list(set(itertools.chain.from_iterable(x_network_edge_list_0)))
x_network_edge_list_1 = [(i,j) for i,j in x_network if x_network[i,j].value == 1]
x_network_node_list_1 = list(set(itertools.chain.from_iterable(x_network_edge_list_1)))

#Prepare network defence visualisation
w_1 = [(i,j) for i,j in w if w[i,j].value == 1]
# nope_1 = [(i,j) for i,j in w if nope[i,j].value == 1]

#Prepare sysprocess visualisation
y_sysprocess_edge_list_0 = [(i,j) for i,j in y_sysprocess if y_sysprocess[i,j].value == 0]
y_sysprocess_node_list_0 = list(set(itertools.chain.from_iterable(y_sysprocess_edge_list_0)))
y_sysprocess_edge_list_1 = [(i,j) for i,j in y_sysprocess if y_sysprocess[i,j].value >= 1]
y_sysprocess_node_list_1 = list(set(itertools.chain.from_iterable(y_sysprocess_edge_list_1)))

In [ ]:
node_colors_network = ['lightblue' if i in x_network_node_list_1 
                       else 'lightgray' for i in network.nodes()]
edge_colors_network = ['blue' if (i, j) in x_network_edge_list_1
                       else 'red' if (i, j) in w_1
                       else 'lightgray' if (i, j) in greyed_out 
                       else 'black' for (i, j) in network.edges()]
edge_widths_network = [2 if (i, j) in x_network_edge_list_1
                       else 2 if (i, j) in w_1
                       else 1 for (i, j) in network.edges()]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
nx.draw_networkx_nodes(network, pos=pos_network, ax=ax, node_color=node_colors_network,
                       edgecolors='black', node_size=2000)
nx.draw_networkx_labels(network, pos=pos_network, ax=ax, labels=dict(zip(nodes_network, nodes_network)),
                        font_size=20)
nx.draw_networkx_edges(network, pos=pos_network, ax=ax, node_size=2000, arrowsize=25, edge_color=edge_colors_network, width=edge_widths_network)
plt.axis('off')
plt.show()

In [ ]:
node_colors = ['lightblue' if i in y_sysprocess_node_list_1 else 'lightgray'
               for i in sysprocess.nodes()]
edge_colors = ['blue' if (i, j) in y_sysprocess_edge_list_1 else 'black'
               for (i, j) in sysprocess.edges()]
edge_widths = [2 if (i, j) in y_sysprocess_edge_list_1 else 1
               for (i, j) in sysprocess.edges()]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 24))
nx.draw_networkx_nodes(sysprocess, pos=pos_sysprocess, ax=ax, node_color=node_colors,
                       edgecolors='black', node_size=2000)
nx.draw_networkx_labels(sysprocess, pos=pos_sysprocess, ax=ax, labels=dict(zip(nodes_sysprocess, nodes_sysprocess)),
                        font_size=20)
nx.draw_networkx_edges(sysprocess, pos=pos_sysprocess, ax=ax, node_size=2000, arrowsize=25, edge_color=edge_colors, width=edge_widths)
plt.axis('off')
plt.show()

## Solving algorithm

In [ ]:
UB = math.inf#Upper boundary
LB = 0#Lower boundary
optimality_gap = UB - LB
K_max = 10#Iteration limit
epsilon = 0#Tolerance
patched_edges = set()#Defence plan
history_attack_edges = set()#Previous attack plans set
history_attack_plans = {}#Previous attack plans dictionary
S = {}#Temporary

for i in range(1,K_max,1):
    z_LO,x_sysprocess,y_sysprocess,x_network,y_network,b = attacker(nodes_network,edges_network,s_network,t_network,
                        targets_network,att_budget,att_cost,nodes_sysprocess,edges_sysprocess,s_sysprocess,t_sysprocess,
                        sys_input,sys_output,capability_input,capability_output,process,art_edges_t_sysprocess,
                        capacity_sysprocess,targets_sysprocess,graph_relation,history_attack_plans,patched_edges)
    
    new_attack_edges = set([i for i in y_network if y_network[i].value == 1])#Determine attack paths
    history_attack_edges = history_attack_edges.union(new_attack_edges)#Merge new with previous
    for j in y_network:
        S[j] = y_network[j].value#Dictionary of new attacks
    history_attack_plans[i] = S#Append new to previous
    
    if z_LO > LB:
        LB = z_LO
        
    if UB - LB <= epsilon:
        print('Optimum found! Effectivity:',LB,'. Best attack:',new_attack_edges,'. Best defence:',patched_edges,'.')
        break

    z_UP_DAD,x_sysprocess,y_sysprocess,x_network,b,w = defender(nodes_network,edges_network,s_network,t_network,
                        targets_network,att_cost,nodes_sysprocess,edges_sysprocess,s_sysprocess,t_sysprocess,sys_input,
                        sys_output,capability_input,capability_output,process,art_edges_t_sysprocess,
                        art_edges_s_network,capacity_sysprocess,targets_sysprocess,graph_relation,
                        history_attack_edges,def_cost,def_budget)

    patched_edges = set([(i,j) for i,j in w if w[i,j].value == 1])#Determine defence plan
    
    if z_UP_DAD < UB:
        UB = z_UP_DAD
    
    if UB - LB <= epsilon:
        print('Optimum found! Effectivity:',UB,'. Best attack:',new_attack_edges,'. Best defence:',patched_edges,'.')
        break

## Visualize Solution with NetworkX

In [ ]:
#Prepare graph visualisation
greyed_out = [(i,j) for (i,j) in x_network if (i,j) not in history_attack_edges]

#Prepare network attack visualisation
x_network_edge_list_0 = [(i,j) for i,j in x_network if x_network[i,j].value == 0]
x_network_node_list_0 = list(set(itertools.chain.from_iterable(x_network_edge_list_0)))
x_network_edge_list_1 = [(i,j) for i,j in x_network if x_network[i,j].value >= 1]
x_network_node_list_1 = list(set(itertools.chain.from_iterable(x_network_edge_list_1)))

#Prepare network defence visualisation
w_1 = [(i,j) for i,j in w if w[i,j].value == 1]

#Prepare sysprocess visualisation
y_sysprocess_edge_list_0 = [(i,j) for i,j in y_sysprocess if y_sysprocess[i,j].value == 0]
y_sysprocess_node_list_0 = list(set(itertools.chain.from_iterable(y_sysprocess_edge_list_0)))
y_sysprocess_edge_list_1 = [(i,j) for i,j in y_sysprocess if y_sysprocess[i,j].value >= 1]
y_sysprocess_node_list_1 = list(set(itertools.chain.from_iterable(y_sysprocess_edge_list_1)))

In [ ]:
node_colors_network = ['lightblue' if i in x_network_node_list_1 
                       else 'lightgray' for i in network.nodes()]
edge_colors_network = ['blue' if (i, j) in x_network_edge_list_1
                       else 'red' if (i, j) in w_1 
                       else 'lightgray' if (i, j) in greyed_out 
                       else 'black' for (i, j) in network.edges()]
edge_widths_network = [2 if (i, j) in x_network_edge_list_1
                       else 2 if (i, j) in w_1
                       else 1 for (i, j) in network.edges()]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
nx.draw_networkx_nodes(network, pos=pos_network, ax=ax, node_color=node_colors_network,
                       edgecolors='black', node_size=2000)
nx.draw_networkx_labels(network, pos=pos_network, ax=ax, labels=dict(zip(nodes_network, nodes_network)),
                        font_size=20)
nx.draw_networkx_edges(network, pos=pos_network, ax=ax, node_size=2000, arrowsize=25, edge_color=edge_colors_network, width=edge_widths_network)
plt.axis('off')
plt.show()

In [ ]:
node_colors = ['lightblue' if i in y_sysprocess_node_list_1 else 'lightgray'
               for i in sysprocess.nodes()]
edge_colors = ['blue' if (i, j) in y_sysprocess_edge_list_1 else 'black'
               for (i, j) in sysprocess.edges()]
edge_widths = [2 if (i, j) in y_sysprocess_edge_list_1 else 1
               for (i, j) in sysprocess.edges()]

In [ ]:
ig, ax = plt.subplots(figsize=(12, 24))
nx.draw_networkx_nodes(sysprocess, pos=pos_sysprocess, ax=ax, node_color=node_colors,
                       edgecolors='black', node_size=2000)
nx.draw_networkx_labels(sysprocess, pos=pos_sysprocess, ax=ax, labels=dict(zip(nodes_sysprocess, nodes_sysprocess)),
                        font_size=20)
nx.draw_networkx_edges(sysprocess, pos=pos_sysprocess, ax=ax, node_size=2000, arrowsize=25, edge_color=edge_colors, width=edge_widths)
plt.axis('off')
plt.show()